In [1]:
!pip install -q ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.6 MB/s eta 0:00:0000:01


In [2]:
!pip install -q supervision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.4/212.4 kB 7.0 MB/s eta 0:00:00


In [3]:
!pip install "numpy<2.0" --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 103.4 MB/s eta 0:00:0000:010:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.26.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
google-colab 1.0.0 requires jupyter-server==2.14.0, but you have jupyter-server 2.12.5 which is incompatible.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
dopamine-rl 4.1.2 requires gymnasium>=1.0.0, but you have gymnasium 0.29.0 which is incompatible.
jaxlib 0.7.2 requ

In [9]:
!unzip /kaggle/input/seadronesseemot-dataset-video-conversion/videos.zip -d /kaggle/working/videos

Archive:  /kaggle/input/seadronesseemot-dataset-video-conversion/videos.zip
   creating: /kaggle/working/videos/kaggle/working/videos/
  inflating: /kaggle/working/videos/kaggle/working/videos/video_009.mp4  
  inflating: /kaggle/working/videos/kaggle/working/videos/video_005.mp4  
  inflating: /kaggle/working/videos/kaggle/working/videos/video_018.mp4  
  inflating: /kaggle/working/videos/kaggle/working/videos/video_015.mp4  
  inflating: /kaggle/working/videos/kaggle/working/videos/video_007.mp4  
  inflating: /kaggle/working/videos/kaggle/working/videos/video_014.mp4  
  inflating: /kaggle/working/videos/kaggle/working/videos/video_000.mp4  
  inflating: /kaggle/working/videos/kaggle/working/videos/video_013.mp4  
  inflating: /kaggle/working/videos/kaggle/working/videos/video_021.mp4  
  inflating: /kaggle/working/videos/kaggle/working/videos/video_012.mp4  
  inflating: /kaggle/working/videos/kaggle/working/videos/video_019.mp4  
  inflating: /kaggle/working/videos/kaggle/working/

In [10]:
import numpy as np
import cv2
from collections import deque, defaultdict
import supervision as sv
from ultralytics import YOLO
from pathlib import Path
import os
import json
import re

# ✅ 1. NOVEL TRACKER (Unchanged logic)
class NovelByteTrack(sv.ByteTrack):
    def __init__(self, track_thresh=0.5, track_buffer=30, match_thresh=0.8, frame_rate=30):
        super().__init__(
            track_activation_threshold=track_thresh,
            lost_track_buffer=track_buffer,
            minimum_matching_threshold=match_thresh,
            frame_rate=frame_rate
        )
        self.track_low_thresh = 0.1
        self.kalman_filters = {}
        self.track_histories = {}
        self.speed_factor = 0.3
        
    def update_with_detections(self, detections):
        detections = self._drone_preprocessing(detections)
        if len(detections) == 0:
            return detections
        tracked_detections = super().update_with_detections(detections)
        tracked_detections = self._novel_postprocessing(tracked_detections)
        return tracked_detections
    
    def _drone_preprocessing(self, detections):
        if len(detections) == 0: return detections
        boxes = detections.xyxy
        confs = detections.confidence
        w = boxes[:, 2] - boxes[:, 0]
        h = boxes[:, 3] - boxes[:, 1]
        aspect_ratio = w / (h + 1e-6)
        valid_mask = (aspect_ratio < 3.0) & (aspect_ratio > 0.3) & (confs > 0.1)
        if np.sum(valid_mask) == 0: return sv.Detections.empty()
        return sv.Detections(xyxy=boxes[valid_mask], confidence=confs[valid_mask], class_id=detections.class_id[valid_mask])
    
    def _novel_postprocessing(self, detections):
        if len(detections) == 0 or detections.tracker_id is None: return detections
        valid_tracks = detections.tracker_id != -1
        if not np.any(valid_tracks): return detections
        valid_ids = detections.tracker_id[valid_tracks]
        valid_boxes = detections.xyxy[valid_tracks]
        valid_confs = detections.confidence[valid_tracks]
        enhanced = valid_confs.copy()
        for i, tid in enumerate(valid_ids):
            tid = int(tid)
            cx, cy = (valid_boxes[i][0] + valid_boxes[i][2])/2, (valid_boxes[i][1] + valid_boxes[i][3])/2
            kf = self._get_kalman_filter(tid, cx, cy)
            kf.predict()
            kf.correct(np.array([[cx], [cy]], dtype=np.float32))
            self._update_track_history(tid, cx, cy, valid_confs[i])
            enhanced[i] *= self._compute_speed_score(tid)
        detections.confidence[valid_tracks] = enhanced
        return detections
    
    def _get_kalman_filter(self, tid, x, y):
        if tid not in self.kalman_filters:
            kf = cv2.KalmanFilter(4, 2)
            kf.transitionMatrix = np.eye(4, dtype=np.float32); kf.transitionMatrix[0,2]=1; kf.transitionMatrix[1,3]=1
            kf.measurementMatrix = np.eye(2, 4, dtype=np.float32)
            kf.processNoiseCov = np.eye(4, dtype=np.float32) * 0.03
            kf.measurementNoiseCov = np.eye(2, dtype=np.float32) * 0.1
            kf.statePre = np.array([[x], [y], [0], [0]], dtype=np.float32)
            kf.errorCovPre = np.eye(4, dtype=np.float32)
            self.kalman_filters[tid] = kf
        return self.kalman_filters[tid]
    
    def _update_track_history(self, tid, x, y, conf):
        if tid not in self.track_histories: self.track_histories[tid] = deque(maxlen=10)
        self.track_histories[tid].append((x, y, conf))
    
    def _compute_speed_score(self, tid):
        if tid not in self.track_histories or len(self.track_histories[tid]) < 2: return 1.0
        pos = list(self.track_histories[tid])
        d = np.linalg.norm(np.array(pos[-1][:2]) - np.array(pos[-2][:2]))
        return np.clip(1.0 - (d / 15.0) * self.speed_factor, 0.3, 1.0)

# ✅ 2. FIXED REAL-TIME EVALUATOR
class RealTimeEvaluator:
    def __init__(self, iou_threshold=0.5):
        self.iou_threshold = iou_threshold
        self.reset_stats()
        # Structure: self.gt_data[video_id][frame_index] = list of objects
        self.gt_data = defaultdict(lambda: defaultdict(list))
        
    def reset_stats(self):
        self.total_gt = 0
        self.total_fp = 0
        self.total_fn = 0
        self.total_matches = 0
        self.iou_sum = 0.0
        self.id_matches = 0
        self.total_ids = 0

    def load_gt(self, gt_path):
        """Loads COCO JSON and maps image_id -> (video_id, frame_index)"""
        print(f"📂 Loading GT from {gt_path}...")
        with open(gt_path, 'r') as f:
            data = json.load(f)
            
        # Step A: Build Image Map (image_id -> video_id, frame_index)
        image_map = {}
        images = data.get('images', [])
        
        print(f"   - Processing {len(images)} images map...")
        for img in images:
            img_id = img['id']
            # Try getting video_id directly, or parse from file_name
            video_id = img.get('video_id')
            file_name = img.get('file_name', '')
            
            # Fallback: Extract video_id from filename if missing (e.g., "video_12/004.jpg")
            if video_id is None:
                v_match = re.search(r'video_(\d+)', file_name)
                video_id = int(v_match.group(1)) if v_match else 0
                
            # Try getting frame_index directly, or parse from file_name
            frame_index = img.get('frame_index')
            if frame_index is None:
                # Heuristic: assume filename ends in number (e.g. "000134.jpg")
                f_match = re.search(r'(\d+)\.(jpg|png|jpeg)$', file_name)
                frame_index = int(f_match.group(1)) if f_match else 0
            
            image_map[img_id] = (int(video_id), int(frame_index))

        # Step B: Load Annotations using map
        annotations = data.get('annotations', [])
        print(f"   - Mapping {len(annotations)} annotations...")
        
        count = 0
        for ann in annotations:
            img_id = ann['image_id']
            if img_id not in image_map:
                continue
            
            vid_id, frame_idx = image_map[img_id]
            
            # Support both 'bbox' [x,y,w,h] and 'rect'
            bbox = ann.get('bbox', ann.get('rect'))
            if bbox:
                self.gt_data[vid_id][frame_idx].append({
                    'track_id': ann.get('track_id', ann.get('id')), # Use 'id' if 'track_id' missing
                    'bbox': bbox
                })
                count += 1
                
        print(f"✅ Successfully loaded {count} GT objects for {len(self.gt_data)} videos.")

    def compute_iou(self, box1, box2):
        x1, y1, w1, h1 = box1
        x2, y2, w2, h2 = box2
        xi1, yi1 = max(x1, x2), max(y1, y2)
        xi2, yi2 = min(x1+w1, x2+w2), min(y1+h1, y2+h2)
        inter = max(0, xi2-xi1) * max(0, yi2-yi1)
        union = w1*h1 + w2*h2 - inter
        return inter / (union + 1e-6)

    def evaluate_frame(self, video_id, frame_idx, pred_tracks):
        # Retrieve GT for specific video AND frame
        gt_frame = self.gt_data[video_id][frame_idx]
        
        self.total_gt += len(gt_frame)
        if not gt_frame:
            self.total_fp += len(pred_tracks)
            return
        if not pred_tracks:
            self.total_fn += len(gt_frame)
            return
            
        gt_matched = [False] * len(gt_frame)
        pred_matched = [False] * len(pred_tracks)
        
        for i, gt in enumerate(gt_frame):
            best_iou, best_j = 0, -1
            for j, pred in enumerate(pred_tracks):
                if pred_matched[j]: continue
                iou = self.compute_iou(gt['bbox'], pred['bbox'])
                if iou > best_iou and iou > self.iou_threshold:
                    best_iou, best_j = iou, j
            
            if best_j != -1:
                gt_matched[i] = True
                pred_matched[best_j] = True
                self.total_matches += 1
                self.iou_sum += best_iou
                if gt['track_id'] == pred_tracks[best_j]['track_id']:
                    self.id_matches += 1
                self.total_ids += 1
                
        self.total_fn += gt_matched.count(False)
        self.total_fp += pred_matched.count(False)

    def get_metrics(self):
        if self.total_gt == 0: return {'Status': 'No GT found for processed videos'}
        mota = 1 - (self.total_fn + self.total_fp) / self.total_gt
        idf1 = 2*self.id_matches / (self.total_gt + self.total_matches + self.total_fp + 1e-6) # Approx
        return {
            'MOTA': round(mota, 3), 
            'IDF1': round(idf1, 3),
            'GT': self.total_gt, 
            'Matches': self.total_matches,
            'FN': self.total_fn, 
            'FP': self.total_fp
        }

# ✅ 3. INFERENCE LOOP
def run_novel_tracking_inference(model_path, videos_folder, output_folder, gt_path=None):
    model = YOLO(model_path)
    tracker = NovelByteTrack(track_thresh=0.4, track_buffer=30)
    
    evaluator = RealTimeEvaluator()
    if gt_path: evaluator.load_gt(gt_path)

    os.makedirs(output_folder, exist_ok=True)
    video_files = list(Path(videos_folder).glob("*.mp4"))
    
    all_preds = []
    
    for video_path in video_files:
        # Extract video ID safely
        try:
            video_id = int(re.search(r'video_(\d+)', video_path.name).group(1))
        except:
            video_id = 0 # Default if pattern doesn't match
            
        print(f"\n🎬 Processing Video ID: {video_id} ({video_path.name})")
        
        cap = cv2.VideoCapture(str(video_path))
        w, h, fps = int(cap.get(3)), int(cap.get(4)), int(cap.get(5))
        out = cv2.VideoWriter(f"{output_folder}/tracked_{video_path.name}", cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
        
        frame_idx = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret: break
            
            # Inference
            results = model(frame, verbose=False, conf=0.25)[0]
            detections = sv.Detections.from_ultralytics(results)
            tracks = tracker.update_with_detections(detections)
            
            # Prepare Predictions
            frame_preds = []
            if len(tracks) > 0 and tracks.tracker_id is not None:
                for i, tid in enumerate(tracks.tracker_id):
                    if tid == -1: continue
                    bbox = tracks.xyxy[i]
                    frame_preds.append({
                        'track_id': int(tid),
                        'bbox': [float(bbox[0]), float(bbox[1]), float(bbox[2]-bbox[0]), float(bbox[3]-bbox[1])]
                    })
                    
                    # Save for JSON output
                    all_preds.append({
                        'video_id': video_id,
                        'frame_index': frame_idx,
                        'track_id': int(tid),
                        'bbox': frame_preds[-1]['bbox']
                    })

            # Evaluate (Pass video_id AND frame_idx)
            if gt_path:
                evaluator.evaluate_frame(video_id, frame_idx, frame_preds)
            
            # Visualize
            annotated = frame.copy()
            if len(tracks) > 0:
                annotated = sv.BoxAnnotator().annotate(annotated, tracks)
                labels = [f"#{tid}" for tid in tracks.tracker_id]
                annotated = sv.LabelAnnotator().annotate(annotated, tracks, labels=labels)
            
            # Live Metrics
            if gt_path:
                metrics = evaluator.get_metrics()
                text = f"MOTA: {metrics.get('MOTA', 0)} | FN: {evaluator.total_fn} | FP: {evaluator.total_fp}"
                cv2.putText(annotated, text, (10, 30), 0, 0.6, (0, 255, 0), 2)
            
            out.write(annotated)
            frame_idx += 1
            
        cap.release()
        out.release()
        print(f"   --> Done. Frames: {frame_idx}")

    # Save Results
    with open(f"{output_folder}/predictions.json", 'w') as f:
        json.dump({'predictions': all_preds, 'metrics': evaluator.get_metrics() if gt_path else {}}, f)
        
    print("\n✅ Final Metrics:", evaluator.get_metrics() if gt_path else "Skipped")
    return all_preds

if __name__ == "__main__":
    MODEL_PATH = "/kaggle/input/enhanced-yolov8-l-on-seadronessee/pytorch/v1/6/last.pt"
    # Update these paths to match your actual structure
    VIDEOS_FOLDER = "/kaggle/working/videos/kaggle/working/videos"  
    OUTPUT_FOLDER = "/kaggle/working/novel_output"
    GT_PATH = "/kaggle/input/seadronesseemot-dataset-video-conversion/unified_mot_dataset.json"
    
    run_novel_tracking_inference(MODEL_PATH, VIDEOS_FOLDER, OUTPUT_FOLDER, GT_PATH)

📂 Loading GT from /kaggle/input/seadronesseemot-dataset-video-conversion/unified_mot_dataset.json...
   - Processing 35843 images map...
   - Mapping 207132 annotations...
✅ Successfully loaded 207132 GT objects for 20 videos.

🚀 Starting Inference Simulation...

🎬 Processing Video ID: 19 (video_019.mp4)
   --> Done. Frames: 913

🎬 Processing Video ID: 21 (video_021.mp4)
   --> Done. Frames: 7870

🎬 Processing Video ID: 13 (video_013.mp4)
   --> Done. Frames: 291

🎬 Processing Video ID: 7 (video_007.mp4)
   --> Done. Frames: 1001

🎬 Processing Video ID: 18 (video_018.mp4)
   --> Done. Frames: 6028

🎬 Processing Video ID: 14 (video_014.mp4)
   --> Done. Frames: 1726

🎬 Processing Video ID: 15 (video_015.mp4)
   --> Done. Frames: 252

🎬 Processing Video ID: 9 (video_009.mp4)
   --> Done. Frames: 3709

🎬 Processing Video ID: 5 (video_005.mp4)
   --> Done. Frames: 715

🎬 Processing Video ID: 0 (video_000.mp4)
   --> Done. Frames: 410

🎬 Processing Video ID: 8 (video_008.mp4)
   --> Done. F